<a href="https://colab.research.google.com/github/Auvarmf/Riset-MBKM-CNN2023/blob/main/Model_Selection_Manual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import classification_report
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tqdm import tqdm
from google.colab import drive
from sklearn.metrics import accuracy_score
from keras.layers import GlobalAveragePooling2D, Dense, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from math import ceil  # Import fungsi ceil untuk membulatkan ke atas

In [118]:
# Mount Google Drive to access files and folders
drive.mount('/content/MBKMCNN')

Drive already mounted at /content/MBKMCNN; to attempt to forcibly remount, call drive.mount("/content/MBKMCNN", force_remount=True).


In [119]:
class_names = ['dita', 'nadaa']

class_names_label = {class_name: i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

IMAGE_SIZE = (224, 224)

In [120]:
def load_data():
    DIRECTORY = '/content/MBKMCNN/MyDrive/MBKMCNN/model_selection_lanjutan'
    CATEGORY = ['train', 'validation']

    output = []

    for category in CATEGORY:
        path = os.path.join(DIRECTORY, category)
        print(path)
        images = []
        labels = []

        print("Loading {}".format(category))

        for folder in os.listdir(path):
            label = class_names_label[folder]

            for file in os.listdir(os.path.join(path, folder)):
                img_path = os.path.join(os.path.join(path, folder), file)

                image = cv2.imread(img_path)
                image = cv2.resize(image, IMAGE_SIZE)

                images.append(image)
                labels.append(label)

        images = np.array(images, dtype="float32")
        labels = np.array(labels, dtype="int32")

        output.append((images, labels))

    return output

In [121]:
(train_images, train_labels), (test_images, test_labels) = load_data()

/content/MBKMCNN/MyDrive/MBKMCNN/model_selection_lanjutan/train
Loading train
/content/MBKMCNN/MyDrive/MBKMCNN/model_selection_lanjutan/validation
Loading validation


In [122]:
def plot_accuracy_loss(history):
    fig = plt.figure(figsize=(10, 5))

    plt.subplot(221)
    plt.plot(history.history['accuracy'], 'bo--', label='acc')
    plt.plot(history.history['val_accuracy'], 'ro--', label='val_acc')
    plt.title("train_acc vs val_acc")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()

    plt.subplot(222)
    plt.plot(history.history['loss'], 'bo--', label='loss')
    plt.plot(history.history['val_loss'], 'ro--', label='val_loss')
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()
    plt.show()

In [123]:
def create_inception_resnet_model():
    input_tensor = Input(shape=(224, 224, 3))

    # Define InceptionResNetV2 architecture
    base_model = InceptionResNetV2(include_top=False, weights=None, input_tensor=input_tensor, input_shape=(224, 224, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(nb_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    return model

In [124]:
model = create_inception_resnet_model()

tf.config.run_functions_eagerly(True)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [125]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_datagen.fit(train_images)

In [126]:
# Hitung steps_per_epoch sebagai pembulatan ke atas dari panjang train_images dibagi dengan batch_size
steps_per_epoch = ceil(len(train_images) / 128)

history = model.fit(train_datagen.flow(train_images, train_labels, batch_size=128),
                    steps_per_epoch=steps_per_epoch,
                    epochs=10,
                    validation_data=(test_images, test_labels))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 [==============================] - 26s 26s/step - loss: 0.7718 - accuracy: 0.4615 - val_loss: 1.1785 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 23s 23s/step - loss: 0.5961 - accuracy: 0.6154 - val_loss: 1.2184 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 22s 22s/step - loss: 0.4500 - accuracy: 0.7692 - val_loss: 1.2336 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 24s 24s/step - loss: 0.3036 - accuracy: 1.0000 - val_loss: 1.1175 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 25s 25s/step - loss: 0.2924 - accuracy: 0.8462 - val_loss: 1.1154 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 25s 25s/step - loss: 0.2022 - accuracy: 0.9231 - val_loss: 1.1142 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 25s 25s/step - loss: 0.2936 - accuracy: 0.8462 - val_loss: 1.0923 - val_accuracy: 0.5000
Epoch 8/10
1/1 [======================

In [127]:
test_loss = model.evaluate(test_images, test_labels)
print("Test loss: {:.4f}".format(test_loss[0]))
print("Test accuracy: {:.4f}".format(test_loss[1]))

1/1 [==============================] - 2s 2s/step - loss: 1.0218 - accuracy: 0.5000
Test loss: 1.0218
Test accuracy: 0.5000


In [128]:
predictions = model.predict(test_images)
pred_labels = np.argmax(predictions, axis=1)
print("Accuracy : {}".format(accuracy_score(test_labels, pred_labels)))

1/1 [==============================] - 3s 3s/step
Accuracy : 0.5


In [129]:
print(classification_report(test_labels, pred_labels, zero_division=1))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         3
           1       1.00      0.00      0.00         3

    accuracy                           0.50         6
   macro avg       0.75      0.50      0.33         6
weighted avg       0.75      0.50      0.33         6

